# Introduce in Q-learning:
Algorithm that allows an agent to learn the optimal action from the environment by using a value-based approach.
 It is an off-policy and model-free algorithm.
By using equation: 

Q(s, a) <---  Q(s, a) + \alpha [ r + \gamma * max_{a'} Q(s', a') - Q(s, a)]

key:
**Grid Size**: The environment is represented as a grid of size 4x4.
**Actions**: The possible actions the agent can take are 'up', 'down', 'left', and 'right'.
**Action Map**: A mapping of action indices to action names.
**Goal State**: The target state the agent aims to reach, represented as coordinates (3,3).
**Alpha (α)**: The learning rate, which determines how much new information overrides the old information.
**Gamma (γ)**: The (discount factor between 0 and 1), which determines the importance of future reward compared to immediate rewards.
**Epsilon (ε)**: The exploration rate, which determines the probability of choosing a random action instead of the best-known action. This decays over time.
**Epsilon Decay**: The rate at which the exploration rate decreases.
**Minimum Epsilon**: The minimum value for the exploration rate.
**Episodes**: The number of episodes the agent will run to learn the optimal policy.

In [57]:
import numpy as np
import random
import time

# Environment setup (4x4 Grid)
GRID_SIZE = 4
ACTIONS = ['up', 'down', 'left', 'right']
ACTION_MAP = {0: 'up', 1: 'down', 2: 'left', 3: 'right'}
GOAL_STATE = (3, 3)
ALPHA = 0.8  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 1.0  # Exploration rate (decays over time)
EPSILON_DECAY = 0.99
MIN_EPSILON = 0.01
EPISODES = 500

# Initialize Q-table
Q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

def get_next_state(state, action):
    """Returns the next state after taking an action."""
    x, y = state
    if action == 'up' and x > 0:
        x -= 1
    elif action == 'down' and x < GRID_SIZE - 1:
        x += 1
    elif action == 'left' and y > 0:
        y -= 1
    elif action == 'right' and y < GRID_SIZE - 1:
        y += 1
    return (x, y)

def get_reward(state):
    """Returns the reward for a given state."""
    return 10 if state == GOAL_STATE else -1

def choose_action(state, epsilon):
    """Chooses an action using an epsilon-greedy policy."""
    if random.uniform(0, 1) < epsilon:
        return random.choice(ACTIONS)  # Exploration
    else:
        x, y = state
        return ACTION_MAP[np.argmax(Q_table[x, y])]  # Exploitation

def train_agent():
    """Trains the Q-learning agent."""
    global EPSILON
    for episode in range(EPISODES):
        state = (0, 0)
        total_reward = 0
        
        while state != GOAL_STATE:
            action = choose_action(state, EPSILON)
            next_state = get_next_state(state, action)
            reward = get_reward(next_state)
            
            # Q-learning update
            x, y = state
            action_idx = ACTIONS.index(action)
            next_x, next_y = next_state
            Q_table[x, y, action_idx] += ALPHA * (reward + GAMMA * np.max(Q_table[next_x, next_y]) - Q_table[x, y, action_idx])
            
            state = next_state
            total_reward += reward
            
        # Decay epsilon
        EPSILON = max(MIN_EPSILON, EPSILON * EPSILON_DECAY)
        if episode % 50 == 0:
            print(f"Episode {episode}: Total Reward = {total_reward}")

def test_agent():
    """Tests the trained Q-learning agent."""
    state = (0, 0)
    steps = []
    while state != GOAL_STATE:
        action = choose_action(state, 0)  # Use greedy policy (epsilon=0)
        steps.append((state, action))
        state = get_next_state(state, action)
    return steps

# Run training and test
time_start = time.time()
train_agent()
time_end = time.time()
print(f"Training completed in {time_end - time_start:.2f} seconds\n")

# Show test run
path = test_agent()
print("Optimal path taken by the agent:")
for step in path:
    print(step)

Episode 0: Total Reward = -37
Episode 50: Total Reward = -2
Episode 100: Total Reward = 2
Episode 150: Total Reward = 4
Episode 200: Total Reward = 5
Episode 250: Total Reward = 5
Episode 300: Total Reward = 5
Episode 350: Total Reward = 5
Episode 400: Total Reward = 5
Episode 450: Total Reward = 5
Training completed in 0.10 seconds

Optimal path taken by the agent:
((0, 0), 'down')
((1, 0), 'down')
((2, 0), 'right')
((2, 1), 'down')
((3, 1), 'right')
((3, 2), 'right')
